In [ ]:
probe_data = pd.read_csv(path + 'all_probes.csv')

av = np.load(path + 'atlas.npy')
probe_data["ccf_entry_ap"]= -probe_data["entry_point_ap"] +5400
probe_data["ccf_entry_ml"]= probe_data["entry_point_rl"] +5700
#probe_data["ccf_entry_dv"]= probe_data["ccfDV"]

probe_data["ap_rounded"] = round(probe_data["ccf_entry_ap"] / 25)
probe_data["ml_rounded"] = round(probe_data["ccf_entry_ml"] / 25) 
av = np.squeeze(av)
for i in range(len(probe_data)):
    ccfAP_rounded = int(probe_data.loc[i,"ap_rounded"])
    ccfML_rounded = int(probe_data.loc[i,"ml_rounded"])
    probe_data.loc[i,'ccf_entry_dv'] = np.where(av[ccfAP_rounded, ccfML_rounded, :] == 1)[0][0] * 25
    
def Rx(t):
    return np.array([[1, 0, 0], [0, np.cos(np.deg2rad(t)), -np.sin(np.deg2rad(t))], [0, np.sin(np.deg2rad(t)), np.cos(np.deg2rad(t))]])

def Ry(t):
    return np.array([[np.cos(np.deg2rad(t)), 0, np.sin(np.deg2rad(t))], [0, 1, 0], [-np.sin(np.deg2rad(t)), 0, np.cos(np.deg2rad(t))]])

def Rz(t):
    return np.array([[np.cos(np.deg2rad(t)), -np.sin(np.deg2rad(t)), 0], [np.sin(np.deg2rad(t)), np.cos(np.deg2rad(t)), 0], [0, 0, 1]])
for i in range(len(probe_data)):
  color = probe_data.loc[i,"color"]
  if color == "red":
    probe_data.loc[i,"color"] = "#EE204D"
  elif color == "orange":
    probe_data.loc[i,"color"] = "#FF7538"
  elif color == "light green":
    probe_data.loc[i,"color"] = "#9ACD32"
  elif color == "dark green":
    probe_data.loc[i,"color"] = "#006400"  ##
  elif color == "medium green":
    probe_data.loc[i,"color"] = "#32CD32"  ##45CEA2
  elif color == "light blue":
    probe_data.loc[i,"color"] = "#1FCECB"  ##
  elif color == "dark blue":
    probe_data.loc[i,"color"] = "#002AFF"  ##
  elif color == "purple":
    probe_data.loc[i,"color"] = "#926EAE"  ##
  elif color == "pink":
    probe_data.loc[i,"color"] = "#D6006E" ##
  elif color == "violet":
    probe_data.loc[i,"color"] = "#8F509D" ##
  else:
    probe_data.loc[i,"color"] = "#000000"
def tip(index):
    horizAngle = probe_data.loc[index,"horizontal_angle"]
    vertAngle = probe_data.loc[index,"vertical_angle"]

    p = np.array([0, 1, 0])  # example value for p

    pRot = np.dot(np.dot(Rz(horizAngle), Rx(-vertAngle)), p.T)  # use -vert because we want to go from +Y towards -Z
    pCCF = np.array([-pRot[1], -pRot[2], pRot[0]])
    ccfEntry = np.array(probe_data.loc[index,["ccf_entry_ap","ccf_entry_dv","ccf_entry_ml"]])
    tip_nick = ccfEntry + pCCF * probe_data.loc[index,"distance_advanced"]  # assuming ccfEntry and distAdvanced are defined
    probe_data.loc[index,"ml"] = tip_nick[2]
    probe_data.loc[index,"ap"] = tip_nick[0]
    probe_data.loc[index,"dv"] = tip_nick[1]
probe_data.index.map(tip)
probe_data['theta'] = probe_data.loc[:,"vertical_angle"] - 90
probe_data['phi'] = probe_data.loc[:,"horizontal_angle"] - 270

In [ ]:
df = pd.DataFrame()
df = probe_data[["session","ml", "ap", "dv", "phi", "theta", "color"]]
df.to_csv(path + 'calculated_probe_data.csv', index=False)